In [305]:
import pandas as pd
import matplotlib

In [306]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [307]:
answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
data=data.merge(answer_metadata, on="AnswerId", how="left")
data=data.groupby("QuestionId")[["Confidence","IsCorrect"]].mean()
# data.dropna(inplace=True)
data['Confidence'].fillna(value=data['Confidence'].mean(),inplace=True)
data['IsCorrect'].fillna(value=data['IsCorrect'].mean(),inplace=True)

from sklearn import preprocessing
data['Confidence']/=100
print(data)
# scaler = preprocessing.StandardScaler().fit(data)
# print(scaler.mean_,scaler.var_)
# scaler = preprocessing.Normalizer().fit(data)
# data = pd.DataFrame(scaler.transform(data),columns=['Confidence','IsCorrect'])
scaler = preprocessing.MinMaxScaler().fit(data)
data = pd.DataFrame(scaler.transform(data),columns=['Confidence','IsCorrect'])

print(data)
print(data['Confidence'].min(),data['Confidence'].max())
print(data['IsCorrect'].min(),data['IsCorrect'].max())

            Confidence  IsCorrect
QuestionId                       
0             0.661765   0.443457
1             0.740362   0.571429
2             0.662014   0.385214
3             0.850212   0.808757
4             0.593434   0.401408
...                ...        ...
943           0.837937   0.566528
944           0.698529   0.142857
945           0.722447   0.422336
946           0.720588   0.459459
947           0.752404   0.617124

[948 rows x 2 columns]
     Confidence  IsCorrect
0      0.549020   0.350699
1      0.653816   0.500000
2      0.549351   0.282750
3      0.800283   0.776884
4      0.457912   0.301643
..          ...        ...
943    0.783916   0.494283
944    0.598039   0.000000
945    0.629930   0.326059
946    0.627451   0.369369
947    0.669872   0.553312

[948 rows x 2 columns]
0.0 1.0
0.0 0.9999999999999999


In [308]:

# data=data.sort_values("Confidence",ascending=True).reset_index()
# data=data.sort_values("Confidence",ascending=False).reset_index()
# data=data.sort_values("IsCorrect",ascending=True).reset_index()
# data=data.sort_values("IsCorrect",ascending=False).reset_index()

data["Quality"] = data['IsCorrect']*data['Confidence']
data=data.sort_values("Quality",ascending=True).reset_index()
data=data.rename(columns={'index':'QuestionId'})

data


,QuestionId,Confidence,IsCorrect,Quality
0,548,0.000000,0.300000,0.000000
1,944,0.598039,0.000000,0.000000
2,931,0.680247,0.020466,0.013922
3,155,0.727626,0.025572,0.018607
4,718,0.561404,0.047872,0.026876
...,...,...,...,...
943,315,1.000000,0.740741,0.740741
944,825,0.853626,0.869781,0.742467
945,828,0.876645,0.852924,0.747712
946,422,0.906250,0.878472,0.796115


In [309]:
eval_validation = pd.read_csv('data/test_data/quality_response_remapped_public.csv')
print(len(eval_validation))
eval_validation.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,0,0,909,318,2,2,2,1.0,2
1,16,16,285,609,2,2,2,2.0,2
2,48,48,132,271,1,2,2,2.0,1
3,38,38,704,127,2,2,1,2.0,2
4,33,33,16,586,1,1,2,2.0,2


In [310]:
eval_test = pd.read_csv('data/test_data/quality_response_remapped_private.csv')
print(len(eval_test))
eval_test.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,15,15,145,762,2,2,2,2.0,2
1,31,31,307,208,1,1,1,2.0,1
2,25,25,278,195,1,1,1,1.0,1
3,3,3,13,153,2,2,2,2.0,2
4,40,40,23,100,2,1,1,1.0,1


In [311]:
submission = pd.read_csv('submission/template.csv')
for idx in range(len(data)):
    tmp = data[data['QuestionId']==idx].index
    submission.loc[idx,'ranking'] = 987654321+idx if tmp.empty else tmp[0]
submission['ranking']=submission['ranking'].astype('int')
submission.to_csv('submission/20172605.csv')

In [312]:
def order2preference(ord, QL, QR):
    preference = []
    for idx in range(len(QL)):
        rankL = ord[ord.QuestionId==QL[idx]].ranking.values[0]
        rankR = ord[ord.QuestionId==QR[idx]].ranking.values[0]
        preference.append(1 if rankL<rankR else 2)
    return preference

def evaluate(truth, submission):
    QL = list(truth.left)
    QR = list(truth.right)
    if len(QL) != len(QR):
        message = 'left and right lengths are not the same'
        exit(message)
    print(list(truth["preference"]))
    return sum(x==y for x,y in zip(truth["preference"],order2preference(submission,QL,QR)))/len(QL)

def evaluates(truth, submission):
    return [
        evaluate(truth.rename(columns={"T1_ALR":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T2_CL":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T3_GF":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T4_MQ":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T5_NS":"preference"})[["left","right","preference"]], submission)]


In [313]:
list_vali = evaluates(eval_validation,submission)
print(list(order2preference(submission,list(eval_validation.left),list(eval_validation.right))))
print(list_vali)
print(max(list_vali))

[2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1]
[2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1]
[2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1]
[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]
[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1]
[1, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1]
[0.72, 0.68, 0.76, 0.64, 0.72]
0.76
